In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("loan_data_500.csv")
df['loan_status'] = df['loan_status'].map({'Y': 1, 'N': 0}) if df['loan_status'].dtype == 'object' else df['loan_status']
if df['person_education'].dtype == 'object':
    df['person_education'] = LabelEncoder().fit_transform(df['person_education'])
X = df[['person_education', 'person_income', 'loan_amnt', 'credit_score']].values
y = df['loan_status'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def step_binary(x):
    return 1 if x >= 0 else 0
def step_bipolar(x):
    return 1 if x >= 0 else -1
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def bipolar_sigmoid(x):
    return (2 / (1 + np.exp(-x))) - 1

class Perceptron:
    def __init__(self, learning_rate=0.01, epochs=1000, activation="step_binary"):
        self.lr = learning_rate
        self.epochs = epochs
        self.activation = activation
    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0
        for _ in range(self.epochs):
            for xi, target in zip(X, y):
                net = self.net_input(xi)
                pred = self.activate(net)
                update = self.lr * (target - pred)
                self.weights += update * xi
                self.bias += update
    def net_input(self, X):
        return np.dot(X, self.weights) + self.bias

    def activate(self, x):
        if self.activation == "step_binary":
            return step_binary(x)
        elif self.activation == "step_bipolar":
            return step_bipolar(x)
        elif self.activation == "sigmoid":
            return 1 if sigmoid(x) >= 0.5 else 0
        elif self.activation == "bipolar_sigmoid":
            return 1 if bipolar_sigmoid(x) >= 0 else -1
        else:
            raise ValueError("Invalid activation function")

    def predict(self, X):
        if len(X.shape) == 1:
            return self.activate(self.net_input(X))
        return np.array([self.activate(self.net_input(xi)) for xi in X])

for act in ["step_binary", "step_bipolar", "sigmoid", "bipolar_sigmoid"]:
    perceptron = Perceptron(learning_rate=0.01, epochs=1000, activation=act)
    perceptron.fit(X_train, y_train)
    y_pred = perceptron.predict(X_test)
    if act in ["step_bipolar", "bipolar_sigmoid"]:
        y_test_adj = np.where(y_test == 0, -1, 1)
    else:
        y_test_adj = y_test
    acc = accuracy_score(y_test_adj, y_pred)
    prec = precision_score(y_test_adj, y_pred, average='binary', pos_label=1)
    rec = recall_score(y_test_adj, y_pred, average='binary', pos_label=1)
    f1 = f1_score(y_test_adj, y_pred, average='binary', pos_label=1)

    print(f"\nActivation: {act}")
    print("Final Weights:", np.round(perceptron.weights, 2))
    print("Final Bias:", round(perceptron.bias, 2))
    print("Accuracy :", round(acc * 100, 2), "%")
    print("Precision:", round(prec * 100, 2), "%")
    print("Recall   :", round(rec * 100, 2), "%")
    print("F1 Score :", round(f1 * 100, 2), "%")



Activation: step_binary
Final Weights: [ 0.02 -0.01 -0.    0.  ]
Final Bias: -0.01
Accuracy : 46.0 %
Precision: 69.44 %
Recall   : 36.76 %
F1 Score : 48.08 %

Activation: step_bipolar
Final Weights: [-0.    0.    0.02  0.01]
Final Bias: 0.04
Accuracy : 68.0 %
Precision: 68.37 %
Recall   : 98.53 %
F1 Score : 80.72 %

Activation: sigmoid
Final Weights: [ 0.02 -0.01 -0.    0.  ]
Final Bias: -0.01
Accuracy : 46.0 %
Precision: 69.44 %
Recall   : 36.76 %
F1 Score : 48.08 %

Activation: bipolar_sigmoid
Final Weights: [-0.    0.    0.02  0.01]
Final Bias: 0.04
Accuracy : 68.0 %
Precision: 68.37 %
Recall   : 98.53 %
F1 Score : 80.72 %
